# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que relación hay entre precipitaciones y el aumento de la generación de energia eléctrica hidráulica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Debido a la falta de datos a nivel diario de REE por provincia o comunidades autonomas, voy a centrar el analisis a nivel de sistema eléctrico (Peninsula, Baleares, Canarias ,Ceuta y Melilla).

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Requisitos para la ejecución del notebook:

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [6]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

###  Lectura datos AEMET

Para la lectura de datos metereólogicos, vamos a utilizar la libreria aemet, de la que utilizaremos los metodos de las clases Aemet y Estacion para obtener los datos a nivel diario de cada estación meterologica para un rago de fechas.

In [11]:
# Obtenemos la lista de estaciones de medicion de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)


In [12]:
# Lectura de datos de AEMET


# Definimos funciones que vamos a utilizar para leer los datos de aement
def estaciones_prov (prov,lista_estaciones):
    '''Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    for estacion in lista_estaciones:
        if estacion['provincia']==prov.upper():
            lista_id.append(estacion['indicativo'])
    return lista_id

help(estaciones_prov)


def lectura_dias(date_ini,date_end,estaciones):
    '''Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicativo'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(0.1) # para evitar errores por nº de lecturas.
    return valores_diarios;

help(lectura_dias)

# Pasamos los datos ha dataframes para su procesado
estaciones=pd.DataFrame(info_estaciones)

date_ini="2020-12-30T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

wheather=pd.DataFrame(lectura_dias(date_ini,date_end,info_estaciones))

print(max(wheather['fecha']),min(wheather['fecha']))


#Rellenamos los NaN por 0 para precipitaciones. 
#Y para temperatura los NaN por la media entre la termperatura del dia anterior y la temperatura del dia siguiente


Help on function estaciones_prov in module __main__:

estaciones_prov(prov, lista_estaciones)
    Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.

Help on function lectura_dias in module __main__:

lectura_dias(date_ini, date_end, estaciones)
    Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.



KeyboardInterrupt: 

In [9]:
wheather

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2020-12-30,0252D,ARENYS DE MAR,BARCELONA,74,"8,6","0,0","3,7",06:30,"13,4",12:30,24,"1,1","6,7",11:10,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,0252D,ARENYS DE MAR,BARCELONA,74,"9,2","0,0","4,8",04:00,"13,5",13:00,24,"2,8","9,4",13:40,NaN,NaN,NaN,NaN,NaN
2,2020-12-30,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,2","0,0","5,1",07:17,"13,3",12:40,32,"6,1","13,3",03:15,"5,4","1014,1",Varias,"1004,2",01
3,2020-12-31,0076,BARCELONA AEROPUERTO,BARCELONA,4,"8,7","0,0","3,6",06:07,"13,8",12:04,24,"4,2","10,8",13:22,"5,1","1014,1",01,"1006,1",24
4,2020-12-30,0200E,"BARCELONA, FABRA",BARCELONA,408,"5,8","0,0","2,3",Varias,"9,4",12:46,33,"6,7","15,0",04:07,"6,9","964,1",24,"954,9",Varias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2020-12-31,5704B,CAZALLA DE LA SIERRA,SEVILLA,550,"3,2","2,3","-3,4",08:00,"9,9",12:40,30,"1,9","7,2",12:20,NaN,"955,3",10,"949,4",24
83,2020-12-30,5641X,ÉCIJA,SEVILLA,130,"6,0","0,0","-0,5",08:00,"12,5",15:10,21,"1,1","3,9",03:40,NaN,"1007,3",10,"1001,2",01
84,2020-12-31,5641X,ÉCIJA,SEVILLA,130,"6,3","0,0","-1,0",07:50,"13,6",15:30,24,"1,4","10,0",23:30,NaN,"1006,9",10,"1000,0",24
85,2020-12-30,5612B,LA RODA DE ANDALUCÍA,SEVILLA,410,"4,0","0,0","-1,4",08:00,"9,4",13:50,02,"1,7","5,0",13:50,NaN,"971,7",11,"966,5",00


In [18]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')

# Me quedo solo con los distintos sistemas electricos existentes
region_system=region_ree[region_ree['geo_limit']!='ccaa']

region_system

,geo_limit,geo_id
Region,,
peninsular,peninsular,8741
canarias,canarias,8742
baleares,baleares,8743
ceuta,ceuta,8744
melilla,melilla,8745


### Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [165]:
geo_limit=region_system[region_system['geo_id']==8745]['geo_limit']
geo_limit[0]

'melilla'

In [ ]:
def lectura_ree(d_inicio,d_fin,geo_id):
    
    # meter esto en una funcion con su try-exception    
    
    geo_limit=region_system[region_system['geo_id']==geo_id]['geo_limit']
    
    parametros={'start_date':'2020-12-31T00:00',
            'end_date':'2021-01-20T00:00',
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':geo_limit[0],
            'geo_ids':geo_id}
    
    URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

    ree_gen=requests.get(URL_GEN,params=parametros)
    
    return ree_gen
    

In [152]:
# Obtenemos los datos de REE a traves de su API.




parametros={'start_date':'2020-12-31T00:00',
            'end_date':'2021-01-20T00:00',
            'time_trunc':'day',
            'geo_trunc':'electric_system',
            'geo_limit':'canarias',
            'geo_ids':'8742'}

URL_GEN='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

ree_gen=requests.get(URL_GEN,params=parametros)
  
if ree_gen.raise_for_status()!=None:
    print('Error en lectura')


In [157]:
df_ree=pd.DataFrame()
for i in range(10):
    df=pd.json_normalize(ree_gen.json()['included'][i]['attributes'],meta=['title','type'],record_path=['values'])
    df_ree=df_ree.append(df)
    
df_ree.reset_index(inplace=True,drop=True)
df_ree

,value,percentage,datetime,title,type
0,9.902,0.000446,2020-12-31T00:00:00.000+01:00,Hidráulica,Renovable
1,5443.054,0.245243,2020-12-31T00:00:00.000+01:00,Motores diésel,No-Renovable
2,4260.363,0.216781,2021-01-01T00:00:00.000+01:00,Motores diésel,No-Renovable
3,5212.173,0.244999,2021-01-02T00:00:00.000+01:00,Motores diésel,No-Renovable
4,5229.706,0.253130,2021-01-03T00:00:00.000+01:00,Motores diésel,No-Renovable
...,...,...,...,...,...
163,20931.266,1.000000,2021-01-16T00:00:00.000+01:00,Generación total,Generación total
164,19821.599,1.000000,2021-01-17T00:00:00.000+01:00,Generación total,Generación total
165,21840.651,1.000000,2021-01-18T00:00:00.000+01:00,Generación total,Generación total
166,21879.485,1.000000,2021-01-19T00:00:00.000+01:00,Generación total,Generación total
